In [2]:
from replay_memory import ReplayMemory, Transition

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class DQN_Linear_Model(nn.Module):
    def __init__(self,  n_features, n_actions):
        super().__init__()
        self.fc1 = nn.Linear(in_features=n_features, out_features=50)
        #self.fc2 = nn.Linear(512, 256)
        self.output = nn.Linear(in_features=50, out_features= n_actions)

    def forward(self, env_input):
        result = env_input.reshape((-1, self.fc1.in_features))
        result = F.relu(self.fc1(result))
        #result = F.relu(self.fc2(result))
        return self.output(result)


In [4]:
import gym
ENV_NAME = 'MountainCar-v0'
env = gym.make(ENV_NAME)
n_features = env.observation_space.shape[0]
n_actions = env.action_space.n

In [6]:
import numpy as np


def discretization(obs):
    # observation_space low y high nos permiten conocer los límites de nuestro ambiente para los valores de Posicion y Velocidad.
    env_low = np.array([-1.2, -0.07])
    env_high = np.array([0.6, 0.07])

    env_den = (env_high - env_low) / 40
    pos_den = env_den[0]
    vel_den = env_den[1]

    pos_low = env_low[0]
    vel_low = env_low[1]

    pos_scaled = int((obs[0] - pos_low) / pos_den)
    vel_scaled = int((obs[1] - vel_low) / vel_den)

    return pos_scaled, vel_scaled

In [7]:
import random
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

size = 20
rm = ReplayMemory(size)
for i in range(size):    
    state = torch.tensor(np.array([random.uniform(-1.2,0.6),random.uniform(-0.7,0.7)]), dtype=torch.float32, device=DEVICE).unsqueeze(0)    
    action = torch.tensor([[env.action_space.sample()]], device=DEVICE, dtype=torch.long)
    next_state = torch.tensor(np.array([random.uniform(-1.2,0.6),random.uniform(-0.7,0.7)]), dtype=torch.float32, device=DEVICE).unsqueeze(0)    
    reward = torch.tensor([random.uniform(-1,1)], device=DEVICE)           
    done = random.choice([True, False]) 
    rm.add(state, action, reward, done, next_state)